문제 정의 : SVM(support vector machine) 사용하여 타이타닉 생존자(1), 사망(0) 예측

기본 라이브러리 불러오기

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
#한글깨짐방지
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


#[Step 1] 데이터 준비 - Seaborn에서 제공하는 titanic 데이터셋 가져오기

In [ ]:
# load_dataset 함수를 사용하여 데이터 프레임으로 변환
df = sns.load_dataset('titanic')

#[Step 2] 데이터 탐색 및 데이터 전처리

In [ ]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


NaN값 많은 열 아예 삭제

In [ ]:
# NaN값이 많은 deck(객실 데크 위치) 열을 삭제, embarked와 내용이 겹치는 embark_town(승선 도시) 열을 삭제

rdf = df.drop(['deck','embark_town'], axis=1)
rdf.info()

null 많은 값 날려버려

In [ ]:
#age는 714건이네? age는 학습할때 중요한정보기때문에, age가 null로 되어 있는 데이터는 날려버리자. = age 열에 나이 데이터가 없는 모든 행을 삭제 -> 177개 NaN 삭제
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    712 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [ ]:
# embarked 열의 NaN값(2건)을 승선도시 중에서 가장 많이 출현한 값 확인 -> 탑승한 도시의 첫글자
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq

'S'

In [ ]:
# fillna 함수를 통해서 embarked 열의 NaN값(2건)을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
rdf['embarked'].fillna(most_freq, inplace=True)

In [ ]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    714 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


#[Step 3] 분석에 사용할 속성(열)을 선택


In [ ]:
# 분석에 활용할 열(속성)을 선택 (생존 여부, 객실 등급, 성별, 나이, 타이타닉에 탑승한 형제/자매 수, 타이타닉에 탑승한 부모/자녀 수, 탑승한 곳(항구) C=Cherbourg, Q=Queenstown, S=Southampton)

ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [ ]:
ndf.info()
# 여기서 문제는 2번 sex가 object로 되어 있어서 컴퓨터가 알아들을수가 없다.
# 6번 embarked 는 C S Q도 마찬가지다.
# 원핫 인코딩해서 sex하고 embarked를 통해서 숫자 0,1 / 0,1,2 로 만들어 버리면 된다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  714 non-null    int64  
 1   pclass    714 non-null    int64  
 2   sex       714 non-null    object 
 3   age       714 non-null    float64
 4   sibsp     714 non-null    int64  
 5   parch     714 non-null    int64  
 6   embarked  714 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 44.6+ KB


In [ ]:
#원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변경
# male, female -> [1,0], [0,1] 으로 만들어 버린다.
# embarked C, Q, S -> [1,0,0], [0,1,0], [0,0,1] 으로 만들어 버린다.

onehot_sex = pd.get_dummies(ndf['sex']) #get_dummies 라는 함수를 쓴다.
onehot_sex

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [ ]:
ndf = pd.concat([ndf, onehot_sex], axis=1)
ndf

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1
...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,Q,1,0
886,0,2,male,27.0,0,0,S,0,1
887,1,1,female,19.0,0,0,S,1,0
889,1,1,male,26.0,0,0,C,0,1


In [ ]:
# embarked C, Q, S -> [1,0,0] , [0,1,0] , [0,0,1]

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [ ]:
#기존 컬럼 삭제
ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head()

# 왜 여자면 0 남자는 1 로 안하고 feature들을 따로 빼주는가?
# 왜냐면, 같은 컬럼안에 여자는0 남자는1 이면, 자료를 찾아내기 쉽지 않을 수 있다.

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


#[Step 4] 데이터셋 구분 - 훈련용(train data) / 검증용 (test data)

In [ ]:
# 속성(변수) 선택
# ndf -> X,y (문제집, 정답지)
X = ndf[['pclass','age',	'sibsp',	'parch',	'female',	'male',	'town_C',	'town_Q',	'town_S']] #독립변수 X

y = ndf['survived'] #종속변수 y

In [27]:
# 설명 변수(데이터,문제집) 정규화(normalization) -> 스케일 조정 0~1사이의 작은 값으로 축소
# 이것을 하는 이유는? age떄문이었다. age가 0~99였으니까 너무 크자나? 그래서 그 값을 0~1로 축소시켜주는 scale을 조정하는 함수를 쓴 것이다.

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [31]:
# train 데이터와 test 데이터로 구분 ( 7:3 비율)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=7)
print("train data 개수 : ", X_train.shape) # 714데이터에서 7:3 비율로 분리 -> 499 :215
print("test data 개수 : ", X_test.shape)

train data 개수 :  (499, 9)
test data 개수 :  (215, 9)


#[Step 5] SVM 분류 모형 - sklearn 사용

In [32]:
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
# 벡터 공간을 맵핑 하는 함수 -> radial basis function (RBF) 커널을 사용
# 선형(linear), 다항식(poly), 가우시안 RBF(rbf) , 시그모이드(sigmoid)

svm_model = svm.SVC(kernel='rbf') #각 공간에다가 feature들을 넣는다. # 벡터 공간을 맵핑 하는 함수 -> 기본 rbf 사용


In [33]:
# 훈련 데이터를 사용하여 모형 학습
svm_model.fit(X_train, y_train)

# 테스트 데이터를 사용하여 y_hat 예측
y_hat = svm_model.predict(X_test)

#[step 6] 모형 성능 평가

In [35]:
from sklearn import metrics

print("테스트 세트 정확도 : {:.2f}%".format(svm_model.score(X_test, y_test)*100))


테스트 세트 정확도 : 80.47%


In [36]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.78      0.94      0.85       126
           1       0.87      0.62      0.72        89

    accuracy                           0.80       215
   macro avg       0.82      0.78      0.79       215
weighted avg       0.82      0.80      0.80       215

